In [13]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, regularizers

### Loading the Data

In [15]:
cvd_data = pd.read_csv(r"Datasets\Analyzed\Analyzed_CVD_Prediction.csv")
cvd_data.head()

,heart_rate,oxygen_saturation,respiratory_rate,temperature,Label
0,93.0,98.0,39.0,36.0,1
1,97.0,98.0,16.0,36.5,0
2,82.0,97.0,25.0,36.3,1
3,98.0,96.0,16.0,36.5,1
4,98.0,99.0,28.0,37.2,1


# Training a DL Model using Keras with 2 Layers

In [16]:
# Split the data into training and validation sets
train_data = cvd_data.sample(frac=0.8, random_state=0)
val_data = cvd_data.drop(train_data.index)

# Preprocess the data
train_labels = train_data.pop('Label')
val_labels = val_data.pop('Label')

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=50,
                    validation_data=(val_data, val_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(val_data, val_labels)

print('Test accuracy:', test_acc)


c:\Users\Sandin Maheeshakya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8459 - loss: 0.3881 - val_accuracy: 0.8712 - val_loss: 0.2001
Epoch 2/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8738 - loss: 0.1988 - val_accuracy: 0.8889 - val_loss: 0.1811
Epoch 3/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8988 - loss: 0.1789 - val_accuracy: 0.9182 - val_loss: 0.1686
Epoch 4/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.9142 - loss: 0.1673 - val_accuracy: 0.9081 - val_loss: 0.1623
Epoch 5/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 913us/step - accuracy: 0.9175 - loss: 0.1624 - val_accuracy: 0.9262 - val_loss: 0.1522
Epoch 6/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.9213 - loss: 0.1535 - val_accuracy: 0.9211 - val_loss: 0.1533
Epoch 7/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - accuracy: 0.9240 - loss: 0.1498 - val_accuracy: 0.9295 - val_loss: 0.1476
Epoch 8/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - accuracy: 0.9302 - loss: 0.1464 - val_ac

#### These scores seems good to be deploy, so it will be converted into a TFLite model

### Converting this model to TFLite Model

In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('Models/CVD_Classification_Model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\SANDIN~1\AppData\Local\Temp\tmplgecygh4\assets


INFO:tensorflow:Assets written to: C:\Users\SANDIN~1\AppData\Local\Temp\tmplgecygh4\assets


Saved artifact at 'C:\Users\SANDIN~1\AppData\Local\Temp\tmplgecygh4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor_26')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2245400496848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2245293910992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2245400494928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2245423483216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2245423483984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2245423483600: TensorSpec(shape=(), dtype=tf.resource, name=None)
